In [19]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
       # if item.abbrev == 'A' or item.abbrev == 'I':
          #  item.abbrev = '-'
        tickers.append(item)

In [20]:
import pandas as pd

submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [21]:
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [22]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [23]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words
0,Submission,me6ezj,Drinking and Trading don’t mix. I accidentally...,CostantlyLost,Title says it all. Had a few too many drinks o...,NaN,NaN,208,t3_me6ezj,https://www.reddit.com/r/stocks/comments/me6ez...,1298,0.95,[1616818448.0],"[Title, says, it, all, Had, a, few, too, many,...","[Drinking, and, Trading, don, t, mix, I, accid..."
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for..."
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about..."
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,..."


In [24]:
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            #if (ticker.name.split()[0] in body_set) or (ticker.name.split()[0] in title_set):
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                #print("Abbreviation match")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                #print("Name match")
                #print(f"{index}: {ticker.name}")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    if len(tickers_found) == 0:
        submissions_df.at[index, "Tickers"] = ""
    else:
        submissions_df.at[index, "Tickers"]= tickers_found
        print(f"{index}: {tickers_found}")

1: ['ADS', 'BCS', 'DRI', 'HON', 'JP', 'MS', 'NDAQ', 'NIO', 'ORLY', 'REGN', 'RH', 'SGFY', 'TLS', 'U', 'DIA', 'QQQ', 'SPY']
2: ['DG', 'E', 'GOOGL', 'HD', 'SPGI', 'TMO', 'TXN', 'UNP', 'V']
3: ['PS']
4: ['AM', 'AMAT', 'BB', 'BNTX', 'CHWY', 'CIO', 'ENPH', 'ET', 'ETSY', 'FCX', 'FOR', 'GILD', 'KMX', 'KNDI', 'L', 'LB', 'LPL', 'LULU', 'MKC', 'MKC.V', 'MTD', 'MU', 'NDAQ', 'NEXT', 'NWG', 'OPEN', 'PM', 'SE', 'TREE', 'TSLA', 'TWOA', 'U', 'VIAC', 'VIACA', 'WBA']
6: ['FANG', 'PLTR', 'PS']
7: ['MX']
10: ['USD']
11: ['DD']
12: ['VIAC']
13: ['CEO', 'FRO', 'NAT', 'STNG', 'TK', 'TNK']
14: ['ARKK']
15: ['F', 'NIO', 'USD']
16: ['DD']
17: ['AMT', 'CCI', 'CTRE', 'EPRT', 'EQIX', 'INVH', 'NSA', 'PLD', 'PSB', 'REXR', 'REIT']
19: ['EV', 'TSLA', 'VWAGY']
20: ['TWTR']
22: ['GME']
23: ['H', 'IJR', 'IWC', 'IWM', 'SDD']
28: ['CP']
29: ['TD']
30: ['NDAQ', 'TA', 'TWOA']
31: ['X', 'Y']
32: ['TMAC']
34: ['MSFT']
36: ['EV', 'GME', 'TWOA', 'ARKK', 'LIT', 'MSOS', 'YOLO']
38: ['CEO', 'DHT', 'DSX', 'FRO', 'NAT', 'SFL', 'TNP', 

In [25]:
submissions_df = submissions_df.loc[submissions_df['Tickers'] != ""]
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]"
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS]
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY..."
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]"


In [27]:
import nltk
#nltk.download('punkt')

In [9]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_df['negative_score'] = ""
submissions_df['neutral_score'] = ""
submissions_df['positive_score'] = ""
submissions_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in submissions_df.iterrows():
    title = row[2]
    body = row[4]
    eval_text = title + " " + body
    body_words = row[12]
    title_words = row[13]
    tickers = row[14]
    pol_score= sia.polarity_scores(eval_text)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    submissions_df.at[index, 'negative_score'] = pol_score['neg']
    submissions_df.at[index, 'neutral_score'] = pol_score['neu']
    submissions_df.at[index, 'positive_score'] = pol_score['pos']
    submissions_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],0,0.898,0.102,0.5632
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...",0.032,0.879,0.089,0.9998
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]",0.042,0.894,0.064,0.9386
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],0.011,0.878,0.111,0.9626
5,Submission,m4pcde,I'm Looking For Solid Youtuber Recommendations...,Vrozini,If anyone has recommendations on Youtubers tha...,NaN,108,t3_m4pcde,https://www.reddit.com/r/stocks/comments/m4pcd...,81,0.91,[1615701622.0],"[If, anyone, has, recommendations, on, Youtube...","[Im, Looking, For, Solid, Youtuber, Recommenda...",[TSLA],0,0.828,0.172,0.8622


In [10]:
submissions_df['sentiment_label'] = 0
submissions_df.loc[submissions_df['compound_score'] > 0.2, 'sentiment_label'] = 1
submissions_df.loc[submissions_df['compound_score'] < -0.2, 'sentiment_label'] = -1
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],0,0.898,0.102,0.5632,1
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...",0.032,0.879,0.089,0.9998,1
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]",0.042,0.894,0.064,0.9386,1
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],0.011,0.878,0.111,0.9626,1
5,Submission,m4pcde,I'm Looking For Solid Youtuber Recommendations...,Vrozini,If anyone has recommendations on Youtubers tha...,NaN,108,t3_m4pcde,https://www.reddit.com/r/stocks/comments/m4pcd...,81,0.91,[1615701622.0],"[If, anyone, has, recommendations, on, Youtube...","[Im, Looking, For, Solid, Youtuber, Recommenda...",[TSLA],0,0.828,0.172,0.8622,1
6,Submission,m4jqge,Favorite Stock/ETF,Rich-Ad-4240,1. What is your favorite long holding stock?\n...,NaN,219,t3_m4jqge,https://www.reddit.com/r/stocks/comments/m4jqg...,75,0.90,[1615682349.0],"[1, What, is, your, favorite, long, holding, s...","[Favorite, Stock, ETF]","[HMNY, WISH, ARKK]",0.051,0.661,0.288,0.9163,1
7,Submission,m4v276,Where do you monitor for the latest breaking c...,electricp0ww0w,"When it comes to shorting stocks, we know that...",NaN,57,t3_m4v276,https://www.reddit.com/r/stocks/comments/m4v27...,62,0.93,[1615728665.0],"[When, it, comes, to, shorting, stocks, we, kn...","[Where, do, you, monitor, for, the, latest, br...",[TSLA],0.05,0.836,0.114,0.8823,1
8,Submission,m4mghu,This DD on $RKT may interest some of you.,BatmanDontGiveNoShit,There was a DD posted on r/wallstreetbets (I k...,NaN,68,t3_m4mghu,https://www.reddit.com/r/stocks/comments/m4mgh...,52,0.83,[1615691225.0],"[There, was, a, DD, posted, on, r, wallstreetb...","[This, DD, on, RKT, may, interest, some, of, y...","[DD, RKT]",0,0.816,0.184,0.9748,1
9,Submission,m4j2cb,Do people with more capital have an advantage ...,ExpensivePumpkin6,I'm still new and trying to learn but I was cu...,NaN,53,t3_m4j2cb,https://www.reddit.com/r/stocks/comments/m4j2c...,37,0.87,[1615680250.0],"[Im, still, new, and, trying, to, learn, but, ...","[Do, people, with, more, capital, have, an, ad...","[AONE, B]",0.12,0.776,0.104,0.3217,1
10,Submission,m4yyvz,How to invest $180k with mental block from hug...,ClosetPervy,"Long story short, I was stupid enough to sell ...",NaN,98,t3_m4yyvz,https://www.reddit.com/r/stocks/comments/m4yyv...,37,0.73,[1615741246.0],"[Long, story, short, I, was, stupid, enough, t...","[How, to, invest, 180k, with, mental, block, f...",[ATH],0.128,0.758,0.114,-0.8114,-1


In [11]:
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0]
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0]
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0]
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0]
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0]


In [12]:
comments_df['Body_Words'] = comments_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')

In [13]:
comments_df['Tickers'] = ""
for index_c, row_c in comments_df.iterrows():
    for index_s, row_s in submissions_df.iterrows():
        if row_c[1] == row_s[1]:
             comments_df.at[index_c, 'Tickers'] = row_s[14]

In [14]:
comments_df = comments_df.loc[comments_df["Tickers"] != ""]
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP]
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP]
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP]
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP]
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP]


In [15]:
comments_df['negative_score'] = ""
comments_df['neutral_score'] = ""
comments_df['positive_score'] = ""
comments_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in comments_df.iterrows():
    body = row[5]
    tickers = row[11]
    pol_score= sia.polarity_scores(body)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    comments_df.at[index, 'negative_score'] = pol_score['neg']
    comments_df.at[index, 'neutral_score'] = pol_score['neu']
    comments_df.at[index, 'positive_score'] = pol_score['pos']
    comments_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP],0,0.953,0.047,0.296
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP],0.15,0.711,0.139,-0.1027
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP],0.031,0.908,0.062,0.25
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP],0,0.8,0.2,0.3612
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP],0,1,0,0


In [16]:
comments_df['percentile_rank']=comments_df.Score.rank(pct=True)

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,percentile_rank
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP],0,0.953,0.047,0.296,0.998588
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP],0.15,0.711,0.139,-0.1027,1.000000
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP],0.031,0.908,0.062,0.25,0.999529
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP],0,0.8,0.2,0.3612,0.992467
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP],0,1,0,0,0.976930


In [18]:
comments_df['weighted_compound'] = comments_df['percentile_rank'] * comments_df['compound_score']
comments_df

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,percentile_rank,weighted_compound
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP],0,0.953,0.047,0.296,0.998588,0.295582
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP],0.15,0.711,0.139,-0.1027,1.000000,-0.1027
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP],0.031,0.908,0.062,0.25,0.999529,0.249882
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP],0,0.8,0.2,0.3612,0.992467,0.358479
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP],0,1,0,0,0.976930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,Comment,m53jxd,gqyj887,t1_gqygx9s,MatterLover1729,['Though not relevant to my comment on republi...,NaN,False,0,[1615765107.0],"[, Though, not, relevant, to, my, comment, on,...",[UK],0.094,0.816,0.09,-0.1687,0.055320,-0.00933251
3259,Comment,m53jxd,gqy1yy3,t1_gqy1cjs,TheXorzox,['two of the biggest states and the cdc adding...,NaN,True,1,[1615756854.0],"[, two, of, the, biggest, states, and, the, cd...",[UK],0,1,0,0,0.225518,0
3260,Comment,m53jxd,gqyjwqg,t1_gqyj887,TheXorzox,['i understand just fine. When democrats cage ...,NaN,True,2,[1615765486.0],"[, i, understand, just, fine, When, democrats,...",[UK],0,0.847,0.153,0.2023,0.466573,0.0943876
3261,Comment,m53jxd,gqykfp3,t1_gqyjwqg,MatterLover1729,['I understand you cannot read'],NaN,False,1,[1615765781.0],"[, I, understand, you, cannot, read, ]",[UK],0,1,0,0,0.225518,0


In [ ]:
weighted_compound_average = comments_df[['Submission_ID','weighted_compound']].groupby('Submission_ID').mean()

In [ ]:
comments_df['sentiment_label'] = 0
comments_df.loc[comments_df['compound_score'] > 0.2, 'sentiment_label'] = 1
comments_df.loc[comments_df['compound_score'] < -0.2, 'sentiment_label'] = -1
comments_df

In [ ]:
comments_df.loc[comments_df['Submission_ID'] == 'm50u8p']

In [ ]:
compound_average = comments_df[['Submission_ID','sentiment_label']].groupby('Submission_ID').mean()

In [ ]:
compound_average

In [ ]:
submissions_df.head()